<a href="https://colab.research.google.com/github/iwatsuki-yuuki/intern/blob/main/%E6%B1%82%E4%BA%BA%E5%88%86%E6%9E%90%E5%BF%9C%E5%8B%9F%E8%80%85%E7%AE%A1%E7%90%86%E3%82%B7%E3%83%BC%E3%83%88_%EF%BC%88Gradio%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install openai==0.28 gradio matplotlib seaborn japanize_matplotlib PyPDF2

In [9]:
# ---------- 必要なライブラリ ----------
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import openai
import gradio as gr
import japanize_matplotlib
import re
from PyPDF2 import PdfReader
import tempfile

openai.api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

# ---------- PDFからキーワード抽出 ----------
def extract_keywords_from_pdf(job_category, pdf_file):
    reader = PdfReader(pdf_file)
    text = "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
    pattern = rf"【{re.escape(job_category)}】(.*?)(?=【|$)"
    matches = re.findall(pattern, text, flags=re.DOTALL)
    if matches:
        keywords = re.split(r"[,\s\n]+", matches[0])
        return list(filter(None, keywords))
    return []

# ---------- GPT-4 プロンプト＆出力 ----------
def create_prompt(job_category, keywords, good_titles, good_contents):
    return f"""
職種「{job_category}」において、クリック率・応募率を高める求人タイトルと原稿を提案してください。

【有効キーワード】
{', '.join(keywords)}

【応募率が高かったタイトル】
{chr(10).join(['・' + t for t in good_titles])}

【応募率が高かった原稿（一部）】
{chr(10).join(['・' + t for t in good_contents])}

【出力形式】
1. タイトル改善案（3案）
2. 原稿改善ポイント（3つ）
"""

def generate_with_gpt4(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "あなたは求人広告の改善アドバイザーです。"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    return response['choices'][0]['message']['content']

# ---------- 群分類関数 ----------
def classify_group(row, ctr_avg, ar_avg):
    ctr, ar = row['クリック率（CTR）'], row['応募率 (AR)']
    if ctr >= ctr_avg and ar >= ar_avg:
        return 'ベストプラクティス群'
    elif ctr < ctr_avg and ar >= ar_avg:
        return 'タイトル改善群'
    elif ctr >= ctr_avg and ar < ar_avg:
        return '原稿改善群'
    else:
        return '全面改善群'

# ---------- 棒グラフ作成関数 ----------
def make_bar_plot(df, col_list, target_col, title_prefix):
    num_cols = len(col_list)
    rows = 1
    cols = num_cols
    fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 5))
    if cols == 1:
        axes = [axes]
    else:
        axes = axes.flatten()
    for i, col in enumerate(col_list):
        mean_values = df.groupby(col)[target_col].mean().sort_values(ascending=False)
        sns.barplot(x=mean_values.index, y=mean_values.values, ax=axes[i])
        axes[i].set_title(f"{title_prefix} by {col}", fontsize=14)
        axes[i].set_xlabel("")
        axes[i].set_ylabel(title_prefix, fontsize=12)
        axes[i].set_ylim(0, 0.05)
        axes[i].tick_params(axis='x', bottom=False, labelbottom=False)
        axes[i].grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    tmpfile = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
    fig.savefig(tmpfile.name)
    plt.close(fig)
    return tmpfile.name

# ---------- メイン処理関数 ----------
def full_pipeline(applicant_csv, indeed_csv, pdf_file):
    df1 = pd.read_csv(applicant_csv.name)
    df2 = pd.read_csv(indeed_csv.name)
    job_category = df1['職業カテゴリー'].dropna().unique()[0]

    df2 = df2[df2['求人のステータス'] == '募集中']
    df = pd.merge(df1, df2, left_on='会社名', right_on='企業名')

    cols = [
        '求人','仕事内容（仕事内容）','仕事内容（アピールポイント）','仕事内容（求める人材）',
        '仕事内容（勤務時間・曜日）','仕事内容（休暇・休日）','仕事内容（勤務地）',
        '都道府県','市区町村','表示回数','クリック数','応募開始数','応募数',
        'クリック単価（CPC）','応募開始単価（CPAS）','応募単価（CPA）',
        'クリック率（CTR）','応募完了率','応募開始率 (ASR)','応募率 (AR)',
        '費用','企業名'
    ]
    df = df[cols]
    df = df[df['応募率 (AR)'] > 0].copy()

    ctr_avg = df['クリック率（CTR）'].mean()
    ar_avg = df['応募率 (AR)'].mean()
    df['カテゴリ'] = df.apply(lambda r: classify_group(r, ctr_avg, ar_avg), axis=1)

    # 📊 図1：CTR×AR分類
    fig, ax = plt.subplots(figsize=(8, 6))
    colors = {'ベストプラクティス群':'green', 'タイトル改善群':'blue', '原稿改善群':'orange', '全面改善群':'red'}
    for group, color in colors.items():
        subset = df[df['カテゴリ'] == group]
        ax.scatter(subset['クリック率（CTR）'], subset['応募率 (AR)'], label=group, color=color)
    ax.axvline(ctr_avg, color='gray', linestyle='--')
    ax.axhline(ar_avg, color='gray', linestyle='--')
    ax.set_xlabel("クリック率（CTR）")
    ax.set_ylabel("応募率 (AR)")
    ax.set_title("CTR×AR分類")
    ax.legend()
    ax.grid(True)
    tmpfile1 = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
    fig.savefig(tmpfile1.name)
    plt.close(fig)

    # 📊 図2・3：棒グラフ
    target_cols = ['仕事内容（仕事内容）','仕事内容（アピールポイント）','仕事内容（求める人材）']
    ar_bar = make_bar_plot(df, target_cols, '応募率 (AR)', "Average AR")
    ctr_bar = make_bar_plot(df, target_cols, 'クリック率（CTR）', "Average CTR")

    # GPT改善提案
    keywords = extract_keywords_from_pdf(job_category, pdf_file)
    good_titles = df.groupby('求人')['応募率 (AR)'].mean().sort_values(ascending=False).head(3).index.tolist()
    good_contents = []
    for col in target_cols:
        good_contents += df.groupby(col)['応募率 (AR)'].mean().sort_values(ascending=False).head(1).index.tolist()
    prompt = create_prompt(job_category, keywords, good_titles, good_contents)
    gpt_output = generate_with_gpt4(prompt)

    return [tmpfile1.name, ar_bar, ctr_bar], gpt_output, df[['求人','クリック率（CTR）','応募率 (AR)','カテゴリ']]

# ---------- Gradio UI ----------
with gr.Blocks(title="GPT × 求人改善AI") as demo:
    gr.Markdown("## 📁 ファイルアップロード")
    with gr.Row():
        input1 = gr.File(label="応募者管理CSV")
        input2 = gr.File(label="Indeed出力CSV")
        input3 = gr.File(label="職種別キーワードPDF")
    run_btn = gr.Button("Submit")

    gr.Markdown("## 📊 可視化図（分類＋棒グラフ）")
    with gr.Row():
        img1 = gr.Image(label="CTR×AR分類図")
        img2 = gr.Image(label="AR棒グラフ")
        img3 = gr.Image(label="CTR棒グラフ")

    gr.Markdown("## 🧠 GPT改善提案")
    gpt_output = gr.Textbox(label="GPTによる提案", lines=10)

    gr.Markdown("## 📋 求人データ（分類済）")
    table_output = gr.Dataframe(label="求人 × CTR × AR × カテゴリ")

    def process_and_run(app_csv, job_csv, pdf):
        return full_pipeline(app_csv, job_csv, pdf)

    run_btn.click(
        fn=process_and_run,
        inputs=[input1, input2, input3],
        outputs=[[img1, img2, img3], gpt_output, table_output]
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://92de060fd6949cd5cb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
